# Evaluate

In [ ]:
import os
PATH = '../parser_workdir'
OUTPUT_DIR = f'{PATH}/srl_parser_goldpredicate'
OUTPUT_DIR = f'{PATH}/srl_parser'
runs = '01,02,03,04,05,06,07,08,09,10'.split(',')


### Seed experiments

In [ ]:

exps_dir = 'nPc_verbs_randAllExps'
exps = [
'010pc_verbs',
'020pc_verbs',
'030pc_verbs',
'040pc_verbs',
'050pc_verbs',
'100pc_verbs',
]

for exp in exps:
    print(exp)
    EXP = f'{exps_dir}/{exp}'
#     break
    for run in runs:
        rescode=os.system(f'HYDRA_CONFIG_PATH=../configurations/run_evaluate_srlparser.yaml python -m sesame.run_evaluate output_dir={OUTPUT_DIR} exp_dir={EXP} model_run={run}')
        print(f'run:{run}-->{rescode}')
        if rescode!=0:
            break
    if rescode!=0:
            break  

### Augmented Experiments

In [ ]:

exps_dir = 'expanded_nPc_verbs_randAllExps'
exps = [
'010pc_verbs_expanded_nouns-50pc',
'020pc_verbs_expanded_nouns-50pc',
'030pc_verbs_expanded_nouns-50pc',
'040pc_verbs_expanded_nouns-50pc',
'050pc_verbs_expanded_nouns-50pc',
'100pc_verbs_expanded_nouns-50pc',
'010pc_verbs_expanded_lu_roles_nouns-50pc',
'020pc_verbs_expanded_lu_roles_nouns-50pc',
'030pc_verbs_expanded_lu_roles_nouns-50pc',
'040pc_verbs_expanded_lu_roles_nouns-50pc',
'050pc_verbs_expanded_lu_roles_nouns-50pc',
'100pc_verbs_expanded_lu_roles_nouns-50pc'
]

pipeline = "lugold_rolegold_nltk_nolemma_role_stopwords_N2"
pipeline = "lugold_rolegold_nltk_nolemma_N2"


for exp in exps:
    print(exp)
    preds_model = 'xlnet_embs_hypers'
    EXP = f'{exps_dir}/{exp}/{preds_model}/{pipeline}'
    for run in runs:
        rescode=os.system(f'HYDRA_CONFIG_PATH=../configurations/run_evaluate_srlparser.yaml python -m sesame.run_evaluate output_dir={OUTPUT_DIR} exp_dir={EXP} model_run={run}')
        print(f'run:{run}-->{rescode}')
        
for exp in exps:
    preds_model = 'bert'
    EXP = f'{exps_dir}/{exp}/{preds_model}/{pipeline}'
    print(exp)
    for run in runs:
        rescode=os.system(f'HYDRA_CONFIG_PATH=../configurations/run_evaluate_srlparser.yaml python -m sesame.run_evaluate output_dir={OUTPUT_DIR} exp_dir={EXP} model_run={run}')
        print(f'run:{run}-->{rescode}')


# Visualize - Learning Curve over  F1-score

In [ ]:
from lexsub.results_parser import parse_results, ttest_df, ttest_df
from lexsub.results_parser import compute_statistical_measures,  compute_statistical_measures_lr
from lexsub.results_parser import get_results_bertSRL, prettify, prettify2

import os
import sys
from glob import glob
import pandas as pd
pd.set_option('max_colwidth', None)


In [ ]:
# def prettify2(df):

#     df['preds_model'] = df['exp_path'].apply(lambda x: x.split('/')[2] if 'expanded' in x else 'base')
#     df['dataset'] = df['exp_path'].apply(lambda x: x.split('/')[1])
#     df['pipeline'] = df['exp_path'].apply(lambda x: x.split('/')[3] if 'expanded' in x else '')
#     df['sample_size'] = df['dataset'].apply(lambda x: int(x[:3]))

#     df['dataset'] = df['dataset'].apply(lambda x: x.split('pc_')[-1])
    
#     df['dataset'] = df['dataset'].apply(lambda x: x.replace('verbs_expanded_','augmented_'))
#     df['dataset'] = df['dataset'].apply(lambda x: x.replace('nouns_expanded_','augmented_'))

#     df['dataset'] = df['dataset'].apply(lambda x: "nPercentData" if x== 'verbs' or x == 'nouns' else x)


    
#     df['dataset'] = df['dataset'].apply(lambda x: x.replace('_', '-'))
#     df['dataset'] = df['dataset'].apply(lambda x: x.replace('lu', 'lexical unit'))
    
#     df = df.round(2)
    
#     return df



### List of paths to experiments

In [ ]:
seed_exps_dir = "nPc_verbs_randAllExps"

seed_exps = """010pc_verbs
020pc_verbs
030pc_verbs
040pc_verbs
050pc_verbs
100pc_verbs""".split('\n')
seed_exps = [f"{seed_exps_dir}/{exp}" for exp in seed_exps]


expanded_exps_dir = 'expanded_nPc_verbs_randAllExps'

expanded_exps1 = """010pc_verbs_expanded_nouns-50pc
020pc_verbs_expanded_nouns-50pc
030pc_verbs_expanded_nouns-50pc
040pc_verbs_expanded_nouns-50pc
050pc_verbs_expanded_nouns-50pc
100pc_verbs_expanded_nouns-50pc""".split('\n')

expanded_exps2 = """010pc_verbs_expanded_lu_roles_nouns-50pc
020pc_verbs_expanded_lu_roles_nouns-50pc
030pc_verbs_expanded_lu_roles_nouns-50pc
040pc_verbs_expanded_lu_roles_nouns-50pc
050pc_verbs_expanded_lu_roles_nouns-50pc
100pc_verbs_expanded_lu_roles_nouns-50pc""".split('\n')

pipeline1 = 'lugold_rolegold_nltk_nolemma_N2'
pipeline2 = 'lugold_rolegold_nltk_nolemma_role_stopwords_N2'

preds_model = 'xlnet_embs_hypers'
exps11 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps12 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps11.extend(exps12)

preds_model = 'bert'
exps21 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps22 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps21.extend(exps22)

exps = seed_exps
exps.extend(exps11)
exps.extend(exps21)
# ------------------------------------------------------------------------------------------
all_exps = exps

print(len(exps))

## Aggregate results in a dataframe

In [ ]:
# ## calculate mean, std, p-value for Experiment-2: learning curve
# def compute_statistical_measures_lr(df,
#                                   eval_measure="f1",
#                                   verbose=False):
    
#     group_columns = ['preds_model', 'dataset', 'task', 'sample_size']

#     dfg=df.groupby(group_columns, as_index=False).agg(['mean', 'std']).reset_index()
#     dfg = dfg.round(2)
#     ps = []

#     for m, ds, n in zip(dfg['preds_model'], dfg['dataset'], dfg['sample_size']):

#         sample1 = df.loc[(df['preds_model']=='base') & (df['sample_size']==n)][eval_measure]
#         sample2 = df.loc[(df['preds_model']==m) & (df['dataset']==ds) & (df['sample_size']==n)][eval_measure]
#         if verbose:
#             if m!="base":
#                 print(n, m, ds)
#                 print("sample1:", list(sample1))
#                 print("sample2:", list(sample2))
#                 print(ttest(sample1, sample2))
#         ps.append(ttest(sample1, sample2))

#     dfg['p_value'] = ps
#     dfg
#     return dfg


# # seed_df = df.loc[df['preds_model']=='base'].copy()
# # # dfg['p_value'] = dfg.apply(lambda row: ttest(seed_df.loc[seed_df['task']==row['task'].iloc[0]]['f1'], 
# # #                                              df.loc[(df['preds_model']==row['preds_model'].iloc[0])  & (df['task']==row['task'].iloc[0]) & (df['exp']==row['exp'].iloc[0])]['f1']), 
# # #                            axis=1)
# # dfg['p_value'] = dfg.apply(lambda row: ttest_df(row, seed_df, df, verbose=False),
# #                                 axis=1)


In [ ]:
def get_results_bertSRL(exps, output_model_dir, verbose=False):
    df = pd.DataFrame(columns=['exp_path', 'task', 'run#', 'f1', 'pre', 'rec'])
    for exp in exps:
        print(exp)
        for model in glob(f'{output_model_dir}/{exp}/*'):
            print(model)
            try:
                run_num = int(model.split('/')[-1].split('-')[-1])
            except:continue    
            try:
                f1, p, r = parse_results(f'{model}/test-f1.txt')

                task_name = 'argid' 
                df.loc[len(df)] = [exp, task_name, run_num, f1, p, r]
            except Exception as e:
                print(e)
                continue
    return df


### Without gold frames

In [ ]:
PATH = '../parser_workdir'
output_model_dir = f'{PATH}/srl_parser'

df = get_results_bertSRL(all_exps, output_model_dir)
df = prettify2(df)
# df
# columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
# group_columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size']

columns = ['preds_model', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
group_columns = ['preds_model', 'dataset', 'task', 'sample_size']

df1 = df[columns]

# check number of runs per dataset/preds_model
dfg=df1.groupby(group_columns, as_index=False).agg(['count']).reset_index()

dfg

In [ ]:
results_df1 = compute_statistical_measures_lr(df1)

### With gold frame

In [ ]:
output_model_dir = f'{PATH}/srl_parser_goldpredicate'
df = get_results_bertSRL(all_exps, output_model_dir)
df = prettify2(df)
# df
# columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
# group_columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size']

columns = ['preds_model', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
group_columns = ['preds_model', 'dataset', 'task', 'sample_size']


df2 = df[columns]

# check number of runs per dataset/preds_model
dfg=df2.groupby(group_columns, as_index=False).agg(['count']).reset_index()
dfg

## Get data in dictionaries for graph

In [ ]:
# ----------------------------
def results_maps(dfg, 
                 base_dataset = 'nPercentData',
                 base_model="base",
                 augmented_datasets = ['augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc'],
                 augmented_models = ['bert', 'xlnet_embs_hypers'],
                 eval_measure="f1"):

    mean_scores = {}
    std_scores = {}
    p_values = {}
#     base model scores
    scores = dfg.loc[(dfg['dataset'] == base_dataset) & (dfg['preds_model'] == base_model)][eval_measure]['mean'].tolist()
    mean_scores[base_model] = {base_dataset:scores}

    scores = dfg.loc[(dfg['dataset'] == base_dataset) & (dfg['preds_model'] == base_model)][eval_measure]['std'].tolist()
    std_scores[base_model] = {base_dataset:scores}

# augmented model scores
    for m in augmented_models:

        all_models_means = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)][eval_measure]['mean'].tolist()
            all_models_means[ds] = scores

        all_models_std = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)][eval_measure]['std'].tolist()
            all_models_std[ds] = scores

        all_models_ps = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)]['p_value'].tolist()
            all_models_ps[ds] = scores
            
        mean_scores[m] = all_models_means
        std_scores[m] = all_models_std
        p_values[m] = all_models_ps
        
    return mean_scores, std_scores, p_values



In [ ]:
results_df1 = compute_statistical_measures_lr(df1)

mean_scores, std_scores, p_values = results_maps(results_df1)
mean_scores, p_values

In [ ]:
# with gold
results_df2 = compute_statistical_measures_lr(df2)

mean_scores_wg, std_scores_wg, p_values_wg = results_maps(results_df2)

mean_scores_wg, p_values_wg

### Learning curve with p-scores

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
%matplotlib inline

# ----------------------------------------------------------------------
N = [10,20,30,40,50, 100]
eval_measure = 'f1'

base_model = 'base'
augmented_models = ['bert', 'xlnet_embs_hypers']

datasets = ['nPercentData', 'augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc']
labels = ['seed', 'augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc']

title = "Learning Curves"
subtitles = {'bert': 'BERT',
            'xlnet_embs_hypers': 'XLNet+embs'}

title = "Learning Curves"
y_label = 'F1-score'
subtitles = {'bert': 'BERT',
            'xlnet_embs_hypers': 'XLNet+embs'}

X_label = "Percentage of training data sampled randomly (dataset: Verbs)"
colors = ['k','c', 'y', 'r', 'brown']
# ----------------------------------------------------------------------
fig, axes = plt.subplots(2, 2, 
                         dpi=480,
                         figsize=(12,8))


# -------------------------------------------------Without gold frames
c = 0
for j, m in enumerate(augmented_models):    
    axes[c][j].set_title(f'Substitution model: {subtitles[m]}')
    axes[c][j].set_xticks(N)
    axes[c][j].set_yticks(range(10, 80, 5))
    axes[c][j].grid()
#     -------------------------------------------
    i = -1
    for ds, label in zip(datasets, labels):
        
        i += 1
        if label == 'seed': model = base_model
        else: 
            model = m   
            ps = p_values_wg[model][ds]
            # ------------------------------- to place P_values on optimally
            exp_names = list(mean_scores_wg[model].keys())
            max_mean = [max(a,b) for a,b in zip(np.array(mean_scores_wg[model][exp_names[0]]), np.array(mean_scores_wg[model][exp_names[-1]]))]
            max_std = [max(a,b) for a,b in zip(np.array(std_scores_wg[model][exp_names[0]]), np.array(std_scores_wg[model][exp_names[-1]]))]
            pY = [x+y for x,y in zip(max_mean, max_std)]
        #     -------------------------------
            
            
        mean = np.array(mean_scores_wg[model][ds])
        std = np.array(std_scores_wg[model][ds])
        axes[c][j].fill_between(N, mean - std,
                             mean + std, alpha=0.1,
                             color=colors[i])

        axes[c][j].semilogx(N, mean, 'o-', color=colors[i],
                 label=label, base=2, nonpositive='clip')
        
#         -------------------------------- add legends for p_values
        pX = N
        p = -1
        if i==1:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 6
                    }
            for xx,yy in zip(pX,pY):
                p += 1
            #   place it below the lines
                if xx in[30, 50, 100]:
                    axes[c][j].text(xx-.5,yy-5, ps[p], fontdict=font) 
            #  place it above the lines
                else:
                    axes[c][j].text(xx-0.75,yy+2, ps[p], fontdict=font) 
        if i==2:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 6
                    }
            for xx,yy in zip(pX,pY):
                p += 1
            #   place it below the lines
                if xx in[30, 50, 100]:
                    axes[c][j].text(xx-0.5,yy-7.5, ps[p], fontdict=font)    
            #  place it above the lines
                else:
                    axes[c][j].text(xx-0.75,yy+4.5, ps[p], fontdict=font) 

                    
# ------------------------------------------------- With gold frames
        
axes[0][c].set_ylabel(f'{y_label} (with gold frames)')
axes[-1][c].set_xlabel(X_label)
c =1

for j, m in enumerate(augmented_models):    
    axes[c][j].set_xticks(N)
    axes[c][j].set_yticks(range(10, 80, 5))
    axes[c][j].grid()
#     -------------------------------------------
    i = -1    
    for ds, label in zip(datasets, labels):
        
        i += 1
        if label == 'seed': model = base_model
        else: 
            model = m 
            ps = p_values[model][ds]
            # ------------------------------- to place P_values on optimally
            exp_names = list(mean_scores_wg[model].keys())
            max_mean = [max(a,b) for a,b in zip(np.array(mean_scores[model][exp_names[0]]), np.array(mean_scores[model][exp_names[-1]]))]
            max_std = [max(a,b) for a,b in zip(np.array(std_scores[model][exp_names[0]]), np.array(std_scores[model][exp_names[-1]]))]
            pY = [x+y for x,y in zip(max_mean, max_std)]
        #     -------------------------------    
            
        mean = np.array(mean_scores[model][ds])
        std = np.array(std_scores[model][ds])
        
        axes[c][j].fill_between(N, mean - std,
                             mean + std, alpha=0.1,
                             color=colors[i])

        axes[c][j].semilogx(N, mean, 'o-', color=colors[i],
                 label=label, base=2, nonpositive='clip')
#         -------------------------------- add legends for p_values  

        pX = N
        p = -1
        if i==1:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 6
                    }
            for xx,yy in zip(pX,pY):
                p += 1
            #   place it below the lines
                if xx in[30, 50, 100]:
                    axes[c][j].text(xx-.5,yy-5, ps[p], fontdict=font) 
            #  place it above the lines
                else:
                    axes[c][j].text(xx-0.75,yy+2, ps[p], fontdict=font) 
        if i==2:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 6
                    }
            for xx,yy in zip(pX,pY):
                p += 1
            #   place it below the lines
                if xx in[30, 50, 100]:
                    axes[c][j].text(xx-0.5,yy-7.5, ps[p], fontdict=font)    
            #  place it above the lines
                else:
                    axes[c][j].text(xx-0.75,yy+4.5, ps[p], fontdict=font) 
#         --------------------------------


axes[1][0].set_ylabel(f'{y_label} (with predicted frames)')
axes[-1][c].set_xlabel(X_label)
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes[:1]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
plt.figlegend( lines, labels, loc = 'lower center', ncol=3, labelspacing=0. , bbox_to_anchor=(0.5, -0.001))
plt.show()